# Домашнее задание «Функции и работа с данными»

In [1]:
import pandas as pd

<div class="alert alert-success">
<h2> Задание 1 </h2>
</div>

Напишите функцию, которая классифицирует фильмы из материалов занятия по следующим правилам:
- оценка 2 и меньше - низкий рейтинг
- оценка 4 и меньше - средний рейтинг
- оценка 5 и меньше - высокий рейтинг

Результат классификации запишите в столбец class

In [2]:
movies = pd.read_csv('ml-latest-small/movies.csv')
movies.columns = ['movie_id', 'title', 'genres']

In [3]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.columns = ['user_id', 'movie_id', 'rating', 'timestamp']

In [4]:
avg_rating = ratings.groupby('movie_id')['rating'].mean().to_frame(name='avg_rating')

In [5]:
def rating_classification(rating):
    if rating <= 2:
        return 'низкий рейтинг'
    if rating <= 4:
        return 'средний рейтинг'
    return 'высокий рейтинг'

In [6]:
avg_rating['class'] = avg_rating['avg_rating'].apply(rating_classification)

In [7]:
movies = movies.merge(avg_rating, on='movie_id')

In [8]:
movies.tail()

,movie_id,title,genres,avg_rating,class
9061,161944,The Last Brickmaker in America (2001),Drama,5.0,высокий рейтинг
9062,162376,Stranger Things,Drama,4.5,высокий рейтинг
9063,162542,Rustom (2016),Romance|Thriller,5.0,высокий рейтинг
9064,162672,Mohenjo Daro (2016),Adventure|Drama|Romance,3.0,средний рейтинг
9065,163949,The Beatles: Eight Days a Week - The Touring Y...,Documentary,5.0,высокий рейтинг


<div class="alert alert-success">
<h2> Задание 2 </h2>
</div>

Используем файл keywords.csv.

Необходимо написать гео-классификатор, который каждой строке сможет выставить географическую принадлежность определенному региону. Т. е. если поисковый запрос содержит название города региона, то в столбце ‘region’ пишется название этого региона. Если поисковый запрос не содержит названия города, то ставим ‘undefined’.

Правила распределения по регионам Центр, Северо-Запад и Дальний Восток:

```
geo_data = {
'Центр': ['москва', 'тула', 'ярославль'],
'Северо-Запад': ['петербург', 'псков', 'мурманск'],
'Дальний Восток': ['владивосток', 'сахалин', 'хабаровск']
}
```
Результат классификации запишите в отдельный столбец region.

In [9]:
keywords = pd.read_csv('ml-latest-small/keywords.csv')

In [10]:
def geo_classification(str):
    if 'москва' in str or 'тула' in str or 'ярославль' in str:
        return 'Центр'
    if 'петербург' in str or 'псков' in str or 'мурманск' in str:
        return 'Северо-Запад'
    if 'владивосток' in str or 'сахалин' in str or 'хабаровск' in str:
        return 'Дальний Восток'
    return 'undefined'

In [11]:
keywords['region'] = keywords['keyword'].apply(geo_classification)

In [12]:
keywords[keywords['region'] == 'Северо-Запад'].head()

,keyword,shows,region
370,авито ру санкт петербург,425134,Северо-Запад
564,погода в санкт петербурге,292489,Северо-Запад
1236,банк санкт петербург,174375,Северо-Запад
1346,санкт петербург,144232,Северо-Запад
2121,электронный дневник санкт петербургского образ...,94835,Северо-Запад


<div class="alert alert-success">
<h2> Задание 3 </h2>
</div>

Есть мнение, что “раньше снимали настоящее кино, не то что сейчас”. Ваша задача проверить это утверждение, используя файлы с рейтингами фильмов из прошлого домашнего занятия (файл ratings.csv из базы https://grouplens.org/datasets/movielens). Т. е. проверить верно ли, что с ростом года выпуска фильма его средний рейтинг становится ниже.

При этом мы не будем затрагивать субьективные факторы выставления этих рейтингов, а пройдемся по следующему алгоритму:

1. В переменную years запишите список из всех годов с 1950 по 2010.

2. Напишите функцию production_year, которая каждой строке из названия фильма выставляет год выпуска. Не все названия фильмов содержат год выпуска в одинаковом формате, поэтому используйте следующий алгоритм:

    - для каждой строки пройдите по всем годам списка years
    - если номер года присутствует в названии фильма, то функция возвращает этот год как год выпуска
    - если ни один из номеров года списка years не встретился в названии фильма, то возвращается 1900 год 


3. Запишите год выпуска фильма по алгоритму пункта 2 в новый столбец ‘year’

4. Посчитайте средний рейтинг всех фильмов для каждого значения столбца ‘year’ и отсортируйте результат по убыванию рейтинга

In [13]:
years = [str(x) for x in range(1950, 2011)]

In [14]:
def production_year(title):
    for year in years:
        if year in title:
            return int(year)
    return 1900

In [15]:
movies['year'] = movies['title'].apply(production_year)

<div class="alert alert-block alert-info">
Если усреднять по годам средние оценки фильмов:
</div>

In [16]:
movies.groupby('year')['avg_rating'].mean().sort_values(ascending=False).head(10)

year
1951    3.906653
1952    3.802318
1962    3.772962
1953    3.738878
1957    3.693322
1973    3.688212
1971    3.643227
1969    3.604509
1972    3.585295
1958    3.571956
Name: avg_rating, dtype: float64

<div class="alert alert-block alert-info">
Если усреднять по годам все оценки:
</div>

In [17]:
ratings.merge(movies, on='movie_id').groupby('year')['rating'].mean().sort_values(ascending=False).head(10)

year
1957    4.014241
1972    4.011136
1952    4.000000
1954    3.994220
1951    3.983539
1974    3.978704
1962    3.952446
1950    3.915254
1977    3.905786
1964    3.841492
Name: rating, dtype: float64